# News Agencies Recognition and Linking with Impresso BERT models

<a target="_blank" href="https://colab.research.google.com/github/impresso/impresso-datalab-notebooks/blob/main/annotate/newsagency-processing_ImpressoHF.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

If something doesn't work, you can [report a problem](https://github.com/impresso/impresso-datalab-notebooks/blob/main/reporting-problems.md).

## What is this notebook about?

This notebook demonstrates how to find mentions of news agencies in historical newspaper articles by loading a pre-trained model from Hugging Face. The model, part of the Impresso Project, is designed to recognize agency names such as Reuters, AFP, Havas, or DPA in texts written in French and German.

It leverages a [language model](https://en.wikipedia.org/wiki/Language_model)-based (more exacly, [BERT](https://en.wikipedia.org/wiki/BERT_(language_model))) multilingual named entity recognition ([NER](https://en.wikipedia.org/wiki/Named-entity_recognition)) model, fine-tuned on manually annotated articles from Swiss and Luxembourgish press (1840–2000), and provides a practical, reproducible way to detect these mentions in your own text collections. More details about how the model and data were created are detailed in the Master thesis of [Marxen, 2023](https://infoscience.epfl.ch/entities/publication/0ca6e53d-d37e-4cdf-a360-2dd9f34a8271).


## Why is this useful?

Delivering swift and reliable news since the 1830s and 1840s, news agencies have played a pivotal role both nationally and internationally. However, understanding their precise influence on shaping historical news content has remained somewhat elusive. Our goal is to illuminate this hidden infrastructure by identifying when and where agency material appeared in the press.

We concentrated on articles we could confidently identify as having agency content, which motivated the following working definition:
> *"An article with news agency content is anything that explicitly cites a news agency as a source."*

This approach ensures high-quality, unambiguous supervision during training — but it also means the model expects mentions to follow certain typical patterns.

#### What counts as an “explicit” mention?

In the training data, agency mentions most commonly appear in specific, identifiable formats, including:
* At the beginning or end of an article:
> ATS-REUTERS, Genève, mardi —
> BERLIN, 10 mars (DPA) –
* As a parenthetical or attributed clause:
> La dépêche vient de (Reuter)
> Selon l’A.F.P., le conflit s’aggrave…
* As a stylistic line break after the dateline:
> LONDRES, 2. — Agence — Les Anglais…

Because these formats appeared more frequently in the dataset, the model is naturally better at detecting them. This is not due to any explicit bias in annotation, but rather a reflection of how agency content was typically presented in 19th–20th century newspaper writing.

#### What the model might not detect

Mentions embedded deep in the article body or framed narratively, like in the following example, are more difficult:

> "Reuters reports from Calcutta that the earthquake threatened part of eastern India..."

Even though this refers to a news agency ("Reuters"), the model might not recognise it as such because:
* There is no explicit label or cue marking it as an agency (e.g., "(EXTEL)" or "—Agence—").
* The mention is part of narrative reporting, not in a typographically distinct format.
* The phrasing may not match surface forms seen frequently enough during training.

#### Best practices for effective use

To achieve the most reliable results:
* Use texts where the agency name appears in a clear, structured format, ideally as part of the byline or first sentence.
* Focus on articles where explicit agency attribution is given, especially through acronyms (e.g., AFP, SDA, DPA).
* Be cautious with more literary or narrative texts, where attribution may be implicit or ambiguous.
* When in doubt, consult [Marxen, 2023](https://infoscience.epfl.ch/entities/publication/0ca6e53d-d37e-4cdf-a360-2dd9f34a8271).


If you're here, you likely seek to detect news agency entities in your own text. This notebook will walk you through how to load the model, apply it to your data, and interpret the output. If your material contains agency mentions in less-standardised forms, the detection of news agencies cannot be ensured.

You can also access our [News Agency Recognition](https://huggingface.co/spaces/impresso-project/multilingual-news-agency-recognition) demo app through [HuggingFace Spaces](https://huggingface.co/docs/hub/en/spaces).

## How it works

This notebook uses a named entity recognition (NER) model built on top of dbmdz/bert-base-historic-multilingual-cased, fine-tuned specifically to identify news agency mentions in historical texts.

The model is applied using the Hugging Face pipeline interface. It performs token classification, labeling each token in the input text with a BIO-style tag that indicates whether it belongs to a recognized agency (e.g., B-Reuters, I-Reuters) or not.

Under the hood, the model has learned to associate known agency names, variants, and historical abbreviations (like "Reuter", "A.F.P.", "dépêche Belga", etc.) with specific tags. It also recognizes more generic cases such as ag (for "agence") and `articleauthor` (for personal bylines).


## What will you learn?

In this notebook, you will:

* Learn how to load and apply a custom NER model from Hugging Face using the transformers library
* Understand how to identify mentions of news agencies in French and German historical newspaper articles
* Explore how the model outputs structured information including entity type, surface form, position, and confidence
* Gain insight into using entity recognition for media provenance analysis in historical datasets

## Useful resources

Here are some resources grouped by level of familiarity, with helpful reminders along the way:

#### Beginner

- [What is Hugging Face?](https://huggingface.co/learn/llm-course/chapter1/1)  
  A simple introduction to the Hugging Face ecosystem and what it offers.

- [Impresso Project Website](https://impresso-project.ch)  
  Overview of the project behind this model, focusing on historical media analysis.

> *"If you don’t understand something now, don't worry — just keep going. It'll often make more sense later."* – [Andrew Ng](https://en.wikipedia.org/wiki/Andrew_Ng)

#### Intermediate

- [Introduction to BERT Models](https://huggingface.co/transformers/model_doc/bert.html)  
  A technical overview of how BERT works and how it’s used in NLP tasks.

- [Named Entity Recognition with Transformers](https://huggingface.co/docs/transformers/main/en/task_summary#token-classification)  
  Explanation of how to apply pretrained transformer models for NER tasks.

> *"Learning is not a linear process. Sometimes, understanding comes after you've seen the bigger picture."* – [Andrew Ng](https://en.wikipedia.org/wiki/Andrew_Ng)

#### Advanced

- [Multilingual News Agency Recognition Demo (HF Spaces)](https://huggingface.co/spaces/impresso-project/multilingual-news-agency-recognition)  
  Explore how the news agency recognition model performs in real-world examples, and try customizing inputs for evaluation.

- [Model Source Code and Training Data](https://github.com/impresso/newsagency-classification)  
  Dive into the codebase used for training and deploying the model, including data preparation and fine-tuning details.

> *"Trust yourself. You understand more than you think you do."* – [Andrew Ng](https://en.wikipedia.org/wiki/Andrew_Ng)

## Prerequisites

Install necessary libraries (if not already installed) and
download the necessary NLTK data.

In [ ]:
!pip install transformers
!pip install stopwordsiso
!pip install nltk

## Basic Usage

To get started, you can load the model directly from Hugging Face using the transformers pipeline. This gives you a simple interface to process texts and extract news agency mentions.

In [35]:
from transformers import pipeline

nlp = pipeline("newsagency-ner", model="impresso-project/ner-newsagency-bert-multilingual", trust_remote_code=True)

text = "REUTERS, Osaka – New severe earthquakes in Japan. Telephone and telegraph connections between Tokyo and Osaka have been disrupted by an earthquake. Trams in Tokyo are at a standstill. Fires have broken out in Sugamo, a Tokyo suburb. A train plunged into the Basubawa, a river between Gotemba and Tokyo. Six trains were overturned. In Yokohama, 600 houses were destroyed."
results = nlp(text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


The output is a list of detected entities with their type, surface form, character offsets, and confidence.

In [27]:
results

[{'type': 'Reuters',
  'confidence': 0.95,
  'index': 0,
  'surface': 'REUTERS',
  'start': 0,
  'end': 7}]

## Advanced Usage

In real-world use cases, especially when working with long articles or batches of OCRed texts, it can be useful to not just extract individual mentions of news agencies but to summarise what agencies appear, how confidently they were detected, and whether the article might be sourced from an agency.

To help with this, we define a simple helper function `analyse_agency_mentions` that takes a full article, uses the model to extract agency mentions, and returns a short analysis including:
* Which agencies were found
* How often each one appears
* The average confidence for each
* A heuristic-based guess on whether the article might contain original news agency content

This is particularly helpful if you're filtering large corpora to identify texts likely to have originated from agencies such as AFP, Havas, Reuters, or DPA.



In [18]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /Users/eboros/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/eboros/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


If you're working with longer articles or entire OCRed pages, it's often best to break the text into sentences or chunks before applying the model. This avoids truncation issues (BERT has a token limit of 512 tokens where [tokens are almost the equivalent of words](https://arxiv.org/pdf/2410.05864)) and improves precision.

In [96]:
# A long historical article (example)
article = """London, 3 September (AFP) – A powerful earthquake struck eastern Japan early this morning. 
Reuters reports that the tremors, which began at 3:57 a.m., caused widespread disruption in Tokyo and Yokohama. 
Fires broke out in the districts of Sugamo and Bunkyo, and several train lines have been suspended.

According to (Reuters), over 600 homes have been destroyed, and dozens are missing or injured. 
AFP correspondents in Yokohama describe scenes of panic, with emergency services struggling to reach the worst-hit areas.

Meanwhile, (DPA) confirms that the German consulate in Yokohama may have suffered structural collapse. 
A bulletin issued by DPA's Kobe bureau states that telegraph lines between Kobe and Tokyo are down, complicating relief efforts.

Further updates are expected later today from (Reuters), (AFP), and (DPA).
"""
# Sentence splitting
sentences = sent_tokenize(article, language="french")

sentences[0]

'London, 3 September (AFP) – A powerful earthquake struck eastern Japan early this morning.'

In [99]:
import numpy as np

def analyse_agency_mentions(sentences, confidence_threshold=0.8):
    """
    Analyse a list of sentences for news agency mentions and print the findings.

    Args:
        sentences (list of str): The input sentences to analyze.
        confidence_threshold (float): Minimum confidence to consider a detection valid.

    Returns:
        details (dict): Dictionary with agency stats and source-likeness scores.
    """
    results = []
    for sentence in sentences:
        results += nlp(sentence)

    if not results:
        print("No news agency mentions found in the text.")
        return {}

    # Collect agency hits
    agency_hits = {}
    for ent in results:
        agency_type = ent.get("type", "unknown")
        confidence = ent.get("confidence", 0.0)
        agency_hits.setdefault(agency_type, []).append(confidence)

    print("This article mentions the following news agencies:")

    total_mentions = sum(len(confidences) for confidences in agency_hits.values())
    details = {}
    source_scores = {}

    for agency, confidences in agency_hits.items():
        count = len(confidences)
        avg_conf = round(np.mean(confidences), 2)
        norm_count = count / total_mentions if total_mentions > 0 else 0
        source_score = round(0.6 * norm_count + 0.4 * avg_conf, 2)

        details[agency] = {
            "occurrences": count,
            "avg_confidence": avg_conf,
            "source_likeness_score": source_score
        }
        source_scores[agency] = source_score
        print(f"- {agency}: {count} occurrence(s), avg. confidence {avg_conf:.2f}, source-likeness score: {source_score}")

    # Identify top candidate
    top_agency = max(source_scores.items(), key=lambda x: x[1])[0]
    top_score = source_scores[top_agency]

    if top_score >= confidence_threshold:
        print(f"✅ This article might be original news agency material from {top_agency}.")
    else:
        print("ℹ️ This article may reference agencies, but is unlikely to be original agency content.")

    return details

In [101]:
details = analyse_agency_mentions(sentences, confidence_threshold=0.6)

This article mentions the following news agencies:
- AFP: 3 occurrence(s), avg. confidence 0.99, source-likeness score: 0.6
- Reuters: 3 occurrence(s), avg. confidence 0.82, source-likeness score: 0.53
- DPA: 3 occurrence(s), avg. confidence 0.97, source-likeness score: 0.59
✅ This article might be original news agency material from AFP.


In [102]:
print(details)

{'AFP': {'occurrences': 3, 'avg_confidence': 0.99, 'source_likeness_score': 0.6}, 'Reuters': {'occurrences': 3, 'avg_confidence': 0.82, 'source_likeness_score': 0.53}, 'DPA': {'occurrences': 3, 'avg_confidence': 0.97, 'source_likeness_score': 0.59}}


The function uses a confidence threshold (default 0.8) to decide whether an article is likely to contain primary agency content. This is based on the idea that original material typically begins with an explicit agency attribution (e.g., "AFP, Berlin, 2 mars – ...").

You can adjust the threshold or modify the logic depending on how conservative or inclusive you want to be in defining "first agency material."


## Conclusion

This notebook provided you with a practical introduction to identifying news agency mentions in historical newspaper articles using a multilingual NER model developed as part of the Impresso project. You learned how to load and apply the model, understand how it interprets explicit agency attributions, and even analyze whether an article might contain first-source agency content.

When using this notebook, keep in mind that the model was trained on texts with explicit, surface-level mentions of agencies (e.g., AFP, DPA, Havas) and may not detect more subtle or implicit attributions. Its performance is best on OCR-cleaned historical articles in French or German, particularly from the 19th and 20th centuries.

Despite these limitations, the tool offers a scalable, reusable method for investigating media sourcing, information flows, and content provenance in digitized historical corpora.

## Next Steps

That’s it for now! If you're interested in expanding your analysis, here are some next steps:

- the [Visualising Place Entities on Maps](https://github.com/impresso/impresso-datalab-notebooks/blob/main/explore-vis/place-entities_map.ipynb) notebook, which demonstrates how to visualise in a map mentions to places in the Impresso corpus.
- Apply this news agency pipeline to your own OCR collections, or combine it with metadata (dates, locations, publishers) to study the historical dynamics of international news coverage.

---
## Project and License info

### Notebook credits [CreditLogo.png](https://credit.niso.org/)
**Writing - Original draft:** Emanuela Boros. **Conceptualization:** Emanuela Boros, Maud Ehrmann. **Software:** Lea Marxen, Emanuela Boros, Maud Ehrmann. **Writing - Review & Editing:** Caio Mello. **Validation:** Kirill Veprikov, Cao Vy. **Datalab editorial board:** Caio Mello (Managing), Pauline Conti, Emanuela Boros, Marten Düring, Juri Opitz, Martin Grandjean, Estelle Bunout, Cao Vy. **Data curation & Formal analysis:** Maud Ehrmann, Emanuela Boros, Pauline Conti, Simon Clematide, Juri Opitz, Andrianos Michail. **Methodology:** Emanuela Boros, Maud Ehrmann. **Supervision:** Emanuela Boros, Maud Ehrmann. **Funding aquisition:** Maud Ehrmann, Simon Clematide, Marten Düring, Raphaëlle Ruppen Coutaz.

<br><a target="_blank" href="https://creativecommons.org/licenses/by/4.0/">
  <img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by.png"  width="100" alt="Open In Colab"/>
</a> 

This notebook is published under [CC BY 4.0 License](https://creativecommons.org/licenses/by/4.0/)

For feedback on this notebook, please send an email to info@impresso-project.ch

### Impresso project

[Impresso - Media Monitoring of the Past](https://impresso-project.ch) is an interdisciplinary research project that aims to develop and consolidate tools for processing and exploring large collections of media archives across modalities, time, languages and national borders. The first project (2017-2021) was funded by the Swiss National Science Foundation under grant No. [CRSII5_173719](http://p3.snf.ch/project-173719) and the second project (2023-2027) by the SNSF under grant No. [CRSII5_213585](https://data.snf.ch/grants/grant/213585) and the Luxembourg National Research Fund under grant No. 17498891.
<br></br>
### License

All Impresso code is published open source under the [GNU Affero General Public License](https://github.com/impresso/impresso-pyindexation/blob/master/LICENSE) v3 or later.


---

<p align="center">
  <img src="https://github.com/impresso/impresso.github.io/blob/master/assets/images/3x1--Yellow-Impresso-Black-on-White--transparent.png?raw=true" width="350" alt="Impresso Project Logo"/>
</p>
